Установить библиотеки: pandas - для работы с таблицами, pyreadstat - для чтения файлов из статистических программ (типа SPSS). Обязательно указать версию библиотеки через ==, потому что код, работающий в одной версии, может не работать в другой.  
Документации на них:

https://pandas.pydata.org

https://ofajardo.github.io/pyreadstat_documentation/_build/html/index.html



In [1]:
!pip install pandas==1.5.3 pyreadstat==1.2.0 tqdm==4.65.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 18.6 MB/s eta 0:00:00


Подключить Google Drive, надо будет дать разрешение этой jupyter-notebook тетрадке на чтение/запись с него.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from collections import defaultdict

import pandas as pd
# импортируем библиотеку pyreadstat, 
# которая ориентирована на работу с форматами статистических (в т.ч., платных) программ.
import pyreadstat
from tqdm.notebook import tqdm as tqdm

In [4]:
fpath_diary_2019 = 'drive/MyDrive/использование_суточного_фонда_времени/BV_DIARYSVODO_2019.sav' # Источник данных: https://gks.ru/ .
df_diary_2019, meta_diary_2019 = pyreadstat.read_file_multiprocessing(pyreadstat.read_sav, fpath_diary_2019, num_processes=16, encoding='cp1251') 
meta_diary_2019.column_names_to_labels

{'H00_02': 'Код субъекта РФ',
 'H00_04': 'Код типа населенного пункта',
 'H00_06': 'Номер домохозяйства в пределах территории',
 'H00_07': 'H00_07',
 'H01_00': 'Индивидуальный код члена домохозяйства',
 'H01_01': 'Пол респондента',
 'H01_02': 'Число исполнившихся лет',
 'D01': 'День недели, в течение которого заполняется дневник',
 'K_110': 'Работа в корпорациях, органах государственного управления и некоммерческих организациях',
 'K_121': 'Выращивание сельскохозяйственных культур на домашних предприятиях для реализации на рынке',
 'K_122': 'Разведение животных на домашних предприятиях для реализации на рынке',
 'K_123': 'Лесоводство и лесозаготовки на домашних предприятиях для реализации на рынке',
 'K_124': 'Ловля рыбы на домашних предприятиях для реализации на рынке',
 'K_125': 'Аквакультура на домашних предприятиях для реализации на рынке',
 'K_126': 'Подземная и открытая разработка полезных ископаемых на домашних предприятиях для реализации на рынке',
 'K_127': 'Производство и пер

In [5]:
selection = [
    'H00_02', # 'Код субъекта РФ',
    'H01_00', # 'Индивидуальный код члена домохозяйства',
    'D01', # 'День недели, в течение которого заполняется дневник',
    'R_1', # 'Место проживания',
    'R_1_1', # 'Размер населенного пункта',
    'H00_06', # 'Номер домохозяйства в пределах территории',


 'K_711', # 'Обсуждения, разговоры, беседы',
 'K_712', # 'Общение и встречи с людьми, деятельность по организации собраний людей',
 'K_713', # 'Чтение и составление почтовых сообщений (включая электронные)',
 'K_719', # 'Другие виды деятельности, связанные с общением и взаимодействием с людьми',

    ] 

In [6]:
def select_variables(df, selection, meta):
    selected = df[selection]
    selection_names = [meta.column_names_to_labels[label].strip() for label in selection]
    selected.columns = selection_names
    return selected

Отличие этих данных от данных КОУЖ: для одного человека может быть более одной строки, так как один и тот же человек заполняет свой дневник в разные дни недели. Поэтому:

1. Группируем по кодам субъекта РФ
2. Группируем по домохозяйствам
3. Группируем по членам домохозяйства
4. Суммируем значения для всех дней недели по интересующим нас переменным
5. Теперь, когда для каждого человека одна строка, можно считать описательные статистики


In [7]:
def select_region(df, code):
    mask = df['Код субъекта РФ'] == code
    region = df[mask]
    print(f"Количество наблюдений по региону {code}: {region.shape[0]}")
    return region

In [8]:
whole_russia_2019 = select_variables(df_diary_2019, selection, meta_diary_2019)
whole_russia_2019 = whole_russia_2019[whole_russia_2019['Место проживания']==1]
my_region_2019 = select_region(whole_russia_2019, '98') # заменить номер региона на свой
my_region_2019.head()

Количество наблюдений по региону 98: 755


,Код субъекта РФ,Индивидуальный код члена домохозяйства,"День недели, в течение которого заполняется дневник",Место проживания,Размер населенного пункта,Номер домохозяйства в пределах территории,"Обсуждения, разговоры, беседы","Общение и встречи с людьми, деятельность по организации собраний людей",Чтение и составление почтовых сообщений (включая электронные),"Другие виды деятельности, связанные с общением и взаимодействием с людьми"
150693,98,1.0,2.0,1.0,1.0,44451.0,20.0,0.0,0.0,0.0
150694,98,1.0,7.0,1.0,1.0,44451.0,0.0,110.0,0.0,0.0
150695,98,1.0,1.0,1.0,1.0,44452.0,0.0,0.0,0.0,0.0
150696,98,1.0,7.0,1.0,1.0,44452.0,60.0,60.0,0.0,0.0
150697,98,1.0,2.0,1.0,1.0,44453.0,10.0,0.0,0.0,0.0


In [9]:
my_region_2019_sums = defaultdict(list)
for household in my_region_2019['Номер домохозяйства в пределах территории'].unique():
    current_household = my_region_2019[my_region_2019['Номер домохозяйства в пределах территории'] == household]
    for member in current_household['Индивидуальный код члена домохозяйства'].unique():
        current_member = current_household[current_household['Индивидуальный код члена домохозяйства'] == member]
        my_region_2019_sums['Размер населенного пункта'].append(current_member['Размер населенного пункта'].iloc[0])
        for column in my_region_2019.columns[6:]:
            my_region_2019_sums[column].append(current_member[column].sum())
my_region_2019_sums = pd.DataFrame(my_region_2019_sums)
# Описательные статистики для городов разного размера данного региона
for size in my_region_2019_sums['Размер населенного пункта'].unique():
    print(meta_diary_2019.value_labels[meta_diary_2019.variable_to_label['R_1_1']][size])
    my_region_2019_sums_by_size = my_region_2019_sums[my_region_2019_sums['Размер населенного пункта']==size]
    for variable in my_region_2019_sums_by_size.columns[1:]:
        print(variable)
        print(my_region_2019_sums_by_size[variable].describe())
    print('----------------------------------------------------------------------------------')

городской, менее 50,0 тыс.чел.
Обсуждения, разговоры, беседы
count    235.000000
mean      55.148936
std       55.480411
min        0.000000
25%       10.000000
50%       40.000000
75%       80.000000
max      290.000000
Name: Обсуждения, разговоры, беседы, dtype: float64
Общение и встречи с людьми, деятельность по организации собраний людей
count    235.000000
mean      46.808511
std       80.789544
min        0.000000
25%        0.000000
50%        0.000000
75%       60.000000
max      380.000000
Name: Общение и встречи с людьми, деятельность по организации собраний людей, dtype: float64
Чтение и составление почтовых сообщений (включая электронные)
count    235.000000
mean       0.340426
std        3.442300
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max       50.000000
Name: Чтение и составление почтовых сообщений (включая электронные), dtype: float64
Другие виды деятельности, связанные с общением и взаимодействием с людьми
count    235.0
mean    

## Всё то же самое, но для 2014 года

In [10]:
fpath_2014 = 'drive/MyDrive/использование_суточного_фонда_времени/BV_Diary_public_2014.sav'
df_2014, meta_2014 = pyreadstat.read_file_multiprocessing(pyreadstat.read_sav, fpath_2014, num_processes=16, encoding='cp1251') 
meta_2014.column_names_to_labels

{'HH_number': 'Номер домохозяйства (сквозной)',
 'D00_08': 'Индивидуальный код члена домохозяйства',
 'BD': 'Будний день',
 'RD': 'Рабочий день',
 'PVD': 'Предвыходной день',
 'VD': 'Выходной день',
 'K_10': 'Рабочее время на основной работе в формальном секторе',
 'K_20': 'Рабочее время на дополнительной работе в формальном секторе',
 'K_30': 'Рабочее время в качестве ученика, стажера, подмастерья в формальном секторе',
 'K_40': 'Короткие перерывы в работе в формальном секторе',
 'K_50': 'Обучение,  профподготовка/переподготовка  в связи с работой в «формальном секторе»',
 'K_80': 'Деятельность, связанная с поиском работы (для желающих работать в формальном секторе)',
 'K_90': 'Организация работы предприятия, деятельность по созданию предприятия в формальном секторе',
 'K_100': 'Передвижения, связанные с работой в «формальном секторе»',
 'K_110': 'Работав «формальномсекторе», неотнесеннаякдругимкатегориям',
 'K_121': 'Освоение земельного участка (основная работа)',
 'K_122': 'Освоение

In [11]:
selection_2014 = [ # набор переменных в 2014 году отличается от 2019
    'D00_08', # 'Индивидуальный код члена домохозяйства',
    'R_1', # 'Место проживания',
    'R_1_1', # 'Размер населенного пункта',
    'HH_number', #'Номер домохозяйства (сквозной)'

 'K_1591', #'Беседы, общение',
 'K_1820', # 'Беседы, общение по телефону',
 'K_1829', # 'Беседы в киберпространстве',
 'K_1830', # 'Общение',
 'K_1840', # 'Чтение и написание писем',
 'K_1850', # 'Чтение и составление электронных почтовых сообщений',
 'K_1860', # 'Антисоциальные/негативные социальные действия',

    ] 
whole_russia_2014 = select_variables(df_2014, selection_2014, meta_2014)
# в данных за 2014 нет кода субъекта РФ
sums = defaultdict(list)

for household in tqdm(whole_russia_2014['Номер домохозяйства (сквозной)'].unique()): 
    current_household = whole_russia_2014[whole_russia_2014['Номер домохозяйства (сквозной)'] == household]
    for member in current_household['Индивидуальный код члена домохозяйства'].unique():
        current_member = current_household[current_household['Индивидуальный код члена домохозяйства'] == member]
        sums['Размер населенного пункта'].append(current_member['Размер населенного пункта'].iloc[0])
        for column in whole_russia_2014.columns[4:]:
            sums[column].append(current_member[column].sum())


  0%|          | 0/9996 [00:00<?, ?it/s]

In [12]:
sums = pd.DataFrame(sums)
for size in sums['Размер населенного пункта'].unique():
    print(meta_2014.value_labels[meta_2014.variable_to_label['R_1_1']][size])
    sums_by_size = sums[sums['Размер населенного пункта']==size]
    for variable in sums_by_size.columns[1:]:
        print(variable)
        print(sums_by_size[variable].describe())
    print('----------------------------------------------------------------------------------')

менее 50,0 тыс. человек
Беседы, общение
count    2829.000000
mean        0.120184
std         3.814990
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max       150.000000
Name: Беседы, общение, dtype: float64
Беседы, общение по телефону
count    2829.000000
mean       14.468010
std        48.828295
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max       660.000000
Name: Беседы, общение по телефону, dtype: float64
Беседы в киберпространстве
count    2829.000000
mean        0.010604
std         0.564033
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max        30.000000
Name: Беседы в киберпространстве, dtype: float64
Общение
count    2829.000000
mean       40.562036
std        83.762323
min         0.000000
25%         0.000000
50%         0.000000
75%        50.000000
max       770.000000
Name: Общение, dtype: float64
Чтение и написание писем
count    2829.000000
mean        